In [56]:
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], True)



import numpy as np
import pandas as pd
import seaborn as sb
sb.set_style("dark")
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

train_df_org = pd.read_csv('kaggle/input/digit-recognizer/train.csv',  nrows=1000)

mnist_train_data = train_df_org.loc[:, "pixel0":]
print(mnist_train_data.head())
# print(mnist_train_data.shape) # (1000, 784)

mnist_train_label = train_df_org.loc[:, "label"]
print(mnist_train_label.head())
# print(mnist_train_label.shape) # (1000,)


   pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   pixel9  ...  pixel774  pixel775  pixel776  pixel777  pixel778  pixel779  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0         0         0         0   
3       0  ...         0         0         0         0         0         0   
4       0  ...         0         0         0         0         0         0   

   pixel780  pixel781  pixel782  pixel783  
0         0         0   

In [57]:


import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras import metrics
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Lambda, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, AvgPool2D
from tensorflow.keras.optimizers import Adadelta
from keras.utils.np_utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler

In [58]:
# Converting dataframe into arrays
mnist_train_data = np.array(mnist_train_data)
mnist_train_label = np.array(mnist_train_label)
# Reshaping the input shapes to get it in the shape which the model expects to recieve later.
mnist_train_data = mnist_train_data.reshape(mnist_train_data.shape[0], 28, 28, 1)
print(mnist_train_data.shape, mnist_train_label.shape)

(1000, 28, 28, 1) (1000,)


In [59]:
nclasses = mnist_train_label.max() - mnist_train_label.min() + 1
mnist_train_label = to_categorical(mnist_train_label, num_classes = nclasses)
print("Shape of ytrain after encoding: ", mnist_train_label.shape)


Shape of ytrain after encoding:  (1000, 10)


In [60]:
def build_model(input_shape=(28, 28, 1)):
    model = Sequential()
    model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 3, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, kernel_size = 4, activation='relu'))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(10, activation='softmax'))
    return model


def compile_model(model, optimizer='adam', loss='categorical_crossentropy'):
    model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


def train_model(model, train, test, epochs, split):
    history = model.fit(train, test, shuffle=True, epochs=epochs, validation_split=split)
    return history

In [61]:
cnn_model = build_model((28, 28, 1))
compile_model(cnn_model, 'adam', 'categorical_crossentropy')

# train the model for as many epochs as you want but I found training it above 80 will not help us and eventually increase overfitting.
model_history = train_model(cnn_model, mnist_train_data, mnist_train_label, 20, 0.2)

Epoch 1/20
25/25 [==============================] - 2s 62ms/step - loss: 2.3521 - accuracy: 0.3150 - val_loss: 1.7041 - val_accuracy: 0.3150
Epoch 2/20
25/25 [==============================] - 1s 55ms/step - loss: 0.8390 - accuracy: 0.7163 - val_loss: 1.5060 - val_accuracy: 0.5500
Epoch 3/20
25/25 [==============================] - 1s 56ms/step - loss: 0.4901 - accuracy: 0.8425 - val_loss: 1.0632 - val_accuracy: 0.6450
Epoch 4/20
25/25 [==============================] - 1s 57ms/step - loss: 0.3548 - accuracy: 0.8875 - val_loss: 0.6909 - val_accuracy: 0.7650
Epoch 5/20
25/25 [==============================] - 1s 56ms/step - loss: 0.2804 - accuracy: 0.9187 - val_loss: 0.4130 - val_accuracy: 0.8650
Epoch 6/20
25/25 [==============================] - 1s 56ms/step - loss: 0.2403 - accuracy: 0.9237 - val_loss: 0.2996 - val_accuracy: 0.8950
Epoch 7/20
25/25 [==============================] - 1s 59ms/step - loss: 0.1724 - accuracy: 0.9463 - val_loss: 0.2041 - val_accuracy: 0.9300
Epoch 8/20
25